# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

In [3]:
## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. 

In [39]:
###### Test pierwszości Rabina-Millera
import random

def rabinMiller(num):
    d = num - 1    ##obliczamy wartości d i sa
    s = 0
    while d % 2 == 0:   ##usuwamy z num-1 dzielniki 2 zliczając je w s
        d = d // 2
        s += 1

    for trials in range(6):   ## wykonujemy n testów R-B 
        a = random.randrange(2, num - 1)  ##losujemy baze a
        b = pow(a, d, num)   ### pierwszy wyraz ciągu
        if (b != 1) and (b != num-1): ## jeśli b nie spełnia warunków losujemy innego świadka
            i = 1
            while (i < s) and (b != (num - 1)):
                b = (b ** 2) % num   ## obliczamy kolejne wyrazy ciągu R-M 
                if(b==1):          ## tylko ostatni wyraz może mieć wartość 1
                    return False 
                i+=1
                
            if(b!=num-1):  ##przedpstatni wyraz musi mieć wartość num -1   
                return False                
    ### jeśli wykonaliśmy n testów i żaden nie zakończył się False 
    return True



def isPrime(num):
    if (num <= 2):
        return False # oczywista oczywistość 
    #opcjonalne można sprawdzić czy małe liczby pierwsze nie są czynnikami badanej liczby
    
    return rabinMiller(num)

#napisz funkcję generującą liczbę pierwszą 


testedNumber=655337

if(isPrime(testedNumber)):
    print(testedNumber, " jest liczbą pierwszą")
else:
    print(testedNumber, " nie jest liczbą pierwszą")

newprime =  generatePrime(16)   
print(isPrime(newprime))


655337  jest liczbą pierwszą
True


## Problem 
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu
3. Oszacuj lub poszukaj informacji o złożoności testu R-M. 

#### Test AKS
Test Agraval-Kayal-Saxena jest pierwszym deterministycznym testem pierwszości (artykuł Primes is in P). 

Sprawdź poprawność działania testu R-M przy pomocy AKS. 

In [5]:
import math

def exp_func(x, y, m):
    exp = bin(y)
    value = x
 
    for i in range(3, len(exp)):
        value = value * value % m
        if(exp[i:i+1]=='1'):
            value = value*x % m
    return value

def phi(n):
    amount = 0
    for k in range(1, n + 1):
        if math.gcd(k, n) == 1:
            amount += 1
    return amount

def step1(n):
    for b in range(2,int(math.log2(n)+1)):
        a = n**(1/b)
        if math.floor(a) == a:
            print("[-]"+str(n)+" is no Prime 1")
            return False
    return True

def step2(n):
    mk = math.log2(n)**2
    nexr = True
    r = 1
    while nexr == True:
        r += 1
        nexr = False
        k = 0
        while k <= mk and nexr == False:
            k = k+1
            if exp_func(n,k,r) == 0 or exp_func(n,k,r) == 1:
                nexr = True
    return r
        

def step3 (n, r):
    for a in range(1,r+1):
        if ((1 < math.gcd(a, n)) and (math.gcd(a,n) < n)):
            print("[-]"+str(n)+" is no Prime 3")
            return False

def step4(n, r):
    if n > 5690034:
        if n <= r:
            print("[+]"+str(n)+" is a Prime Step 4")
            return True


def step5(n, r):
    x = 7
    max = math.sqrt(phi(r))
    rn = math.floor(max*math.log2(n))
    cache = exp_func(x,n,n)
    for a in range(1, rn+1):
        b = exp_func((x+a),n,n) #((x + a) ** n) % n
        l = (cache+a)%n #(x ** n + a) % n
        if b != l:
            print("[-]"+str(n)+" is no Prime 5")
            return False
    print("[+]"+str(n)+" is a Prime Number Step 5")
    return True


def aks(n):
    print("Testing Number: "+str(n))
    if step1(n) == True:
        r = step2(n)
        if step3(n, r) != False:
            if step4(n, r) != True:
                return step5(n, r)


        
#for i in range(2,1000):
#   aks(i)

print(100207100213100237100267*100207100213100237100267)
print(aks(41047))
print(aks(10041462933118313583672041643590011936611471289))
print(aks(42899))
print(aks(25326001))
#aks(671998030559713968361666935769)



10041462933118313583672041643590011936611471289
Testing Number: 41047
[+]41047 is a Prime Number Step 5
True
Testing Number: 10041462933118313583672041643590011936611471289
[-]10041462933118313583672041643590011936611471289 is no Prime 1
None
Testing Number: 42899
[+]42899 is a Prime Number Step 5
True
Testing Number: 25326001
[-]25326001 is no Prime 5
False


## Zastanów się
Dlaczego test Rabina-Millera stosowany jest częściej niż AKS? 

In [ ]:
import pylab
import numpy

x = numpy.linspace(1,30,200) 
#y = numpy. Złożoność testu AKS
w = numpy.linspace(1,30,200) 
#v = numpy. Złożoność testu R-M 

pylab.plot(x,y)
pylab.plot(w,v)
pylab.show() 

#### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$: $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1) * (q-1)$
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)(q-1))$: $de = 1  (mod\ (p-1)(q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

In [6]:
import random, sys, os

def generateKey(keySize):
    print('1. Generujemy liczby p i q')
   

    print('2. Generujemy wykładnik publiczny (względnie pierwszy z (p-1)(q-1))')
    # np. poszukiwanie losowe  
  
    print('3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)')

    publicKey = ()
    privateKey = ()
    print('Klucz publiczny:', publicKey)
    print('Klucz prywatny:', privateKey)
    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(16)
    print(public, private)
  
print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(8)

Generujemy klucze publiczny i prywatny
1. Generujemy liczby p i q
2. Generujemy wykładnik publiczny (względnie pierwszy z (p-1)(q-1))
3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)
Klucz publiczny: ()
Klucz prywatny: ()
() ()


## Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [7]:
def encrypt(message, modulus, exp):
    # kod szyfrowania     
 
    return message_encrypted


keyPublic, keyPrivate = generateKey(8)
print(keyPublic, keyPrivate)

ciphertext= encrypt('Hello world', keyPublic[0], keyPublic[1])
print(ciphertext)

1. Generujemy liczby p i q
2. Generujemy wykładnik publiczny (względnie pierwszy z (p-1)(q-1))
3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)
Klucz publiczny: ()
Klucz prywatny: ()
() ()


IndexError: tuple index out of range

## Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [ ]:
def decrypt(message_encrypted, modulus, exp):
    
   # kod deszyfrowania 

    return ('').join(message)

plaintext = decrypt(ciphertext, keyPrivate[0], keyPrivate[1])
print(plaintext)

## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 

## Zadanie dodatkowe
1. Przeprowadź dowód poprownosci systemu RSA 